# 引越しの口コミサイトをスクレイピング

参考：[複数のページをまたいでスクレイピングしたいです。](https://teratail.com/questions/220626?sort=3)

In [ ]:
!pip install requests
!pip install beautifulsoup4

In [ ]:
import time
import requests
from bs4 import BeautifulSoup
from urllib.parse import urljoin
import pandas as pd
import datetime

date_data = []
compares_data = []
titles_data = []
scores_data = []
comments_data = []



for n in range(1, 300):
  time.sleep(1)
  url = f'https://hikkoshizamurai.jp/company/d-sakaico/voice/p1-com101?voice_page={n}'
  res = requests.get(url)
  soup = BeautifulSoup(res.text,"html.parser")

  # 日付の取得
  tables = soup.find_all("table",attrs={"voice-table"})
  for i in range(len(tables)):
    date = tables[i].text.replace("\n","").replace("引越し時期","").replace("年","-").replace("月","-")
    dt, _ = date.split("日(")
    date_data.append(dt)


  # タイトルの取得
  titles = soup.find_all("div",attrs={"voice-title"})
  for t in titles:
    ts = t.text
    titles_data.append(ts)

  # 評価点の取得
  scores = soup.find_all("div",attrs={"voice-rate-number"})
  for s in scores:
    ss = s.text.replace("点","").replace("評価数：","").replace("\n", "").replace(" ","")
    scores_data.append(float(ss))

  # コメントの取得
  comments = soup.find_all("div",attrs={"voice-comment-text"})
  for c in comments:
    cs = c.text.replace("\n","").replace(" ","").replace("\r","")
    comments_data.append(cs)



data = {"date":1,
        "title":1,
        "rate":1,
        "compare":1}

data["date"] = date_data
data["title"] = titles_data
data["rate"] = scores_data
data["comment"] = comments_data
df = pd.DataFrame(data)

なぜか

data["compare"] = compares_data

```
data["compare"] = compares_data
```
とすると



```
TypeError: 'str' object does not support item assignment
```
となってしまうが、別のセルで実行するとうまくいく


In [ ]:
df.head(10)

- 参考：[Pythonのdatetimeで日付や時間と文字列を変換（strftime, strptime）](https://note.nkmk.me/python-datetime-usage/)

- 参考：[PythonでCSVオブジェクトを日付順で並べ替える
](https://laboratory.kazuuu.net/sorting-csv-objects-by-date-in-python/)

In [ ]:
df["date"] = pd.to_datetime(df["date"])
df.sort_values(by = 'date', ascending = True, inplace = True)
df.head()

In [ ]:
df.to_csv("moving_sq1.csv")